# Sprint 機械学習スクラッチ入門
このSprintでは機械学習手法のスクラッチ課題に取り組む準備を行います。scikit-learnを用いて分類・回帰問題を解くコードを書いておき、今後のSprintではそれと同じ動作をするクラスをスクラッチで作成していきます。

#### スクラッチの意義
ここでのスクラッチとは、NumPyなどの基本的なライブラリを組み合わせることで、scikit-learnのような応用的なライブラリと同じ機能のクラス・関数を自作することを指します。<br>
<br>
スクラッチをすることでscikit-learnなどのライブラリを動かすだけでは掴みづらい、アルゴリズムの深い理解を目指します。コーディングのスキル向上も兼ねますが、それは主な目的ではありません。<br>
<br>
以下のような効果を狙っています。<br>

- 新たな手法に出会った時に理論・数式を理解しやすくする
- ライブラリを使う上での曖昧さを減らす
- 既存の実装を読みやすくする

## 【問題1】train_test_splitのスクラッチ
スクラッチの練習として、scikit-learnのtrain_test_splitを自作してみます。以下の雛形をベースとして関数を完成させてください。<br>
<br>
[sklearn.model_selection.train_test_split — scikit-learn 0.22.2 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)<br>
<br>
なお、作成した関数がscikit-learnのtrain_test_splitと同じ動作をしているか必ず確認をするようにしましょう。

In [84]:
def scratch_train_test_split(X, y, train_size=0.8, stratify=True):
    """
    検証データを分割する。

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, )
      正解値
    train_size : float (0<train_size<1)
      何割をtrainとするか指定
    stratify : bool
      層化の有無を指定

    Returns
    ----------
    X_train : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    X_test : 次の形のndarray, shape (n_samples, n_features)
      検証データ
    y_train : 次の形のndarray, shape (n_samples, )
      訓練データの正解値
    y_test : 次の形のndarray, shape (n_samples, )
      検証データの正解値
    """
    #ここにコードを書く
    
    #stratify機能を追加
    #StratifiedShuffleSplitのソースコードを参考にした
    if stratify:
        labels, y_indices = np.unique(y, return_inverse=True)  #ラベルの種類と値をリストで取得
        label_counts = np.bincount(y_indices)  #各種ラベルのサンプル数を取得
        n_labels = len(labels)  #ラベルの種類数を取得
        
        #全てのサンプルにインデックスを与え、ラベル毎にリスト化
        label_indices = np.split(np.argsort(y_indices, kind='mergesort'),
                                 np.cumsum(label_counts)[:-1])
        
        train = []
        test = []

        #ラベル毎に下記を繰り返す
        for i in range(n_labels):
            #同一ラベル内で取得するサンプルをランダム化
            permutation = np.random.permutation(label_counts[i])  
            perm_indices_label_i = label_indices[i].take(permutation, mode='clip')

            #分割率に合わせて、ラベルのインデックスをtrainとtestに振り分ける
            split_length = round(label_counts[i]*train_size).astype(int)
            train.extend(perm_indices_label_i[:split_length])
            test.extend(perm_indices_label_i[split_length:])
        
        #インデックスに応じたサンプルを格納
        X_train = X[train]
        X_test = X[test]

        y_train = y[train]
        y_test = y[test]

    else:
        split_length = round(len(X)*train_size)
    
        X_train = X[:split_length]
        X_test = X[split_length:]

        y_train = y[:split_length]
        y_test = y[split_length:]
    
    return X_train, X_test, y_train, y_test

In [85]:
# スクラッチの動作確認
import numpy as np
from sklearn.model_selection import train_test_split

# ダミーデータ作成
X = np.zeros((1284, 3))
y = np.zeros((1284, 1))

# スクラッチ関数によるtrainデータ数を確認
X_train, X_test, y_train, y_test = scratch_train_test_split(X, y)

print("スクラッチ版のX_trainデータ数：{}".format(len(X_train)))
print("スクラッチ版のy_trainデータ数：{}".format(len(y_train)))
print("\n")

# sklearn関数によるtrainデータ数を確認
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
print("sklearn版のX_trainデータ数：{}".format(len(X_train)))
print("sklearn版のy_trainデータ数：{}".format(len(y_train)))

スクラッチ版のX_trainデータ数：1027
スクラッチ版のy_trainデータ数：1027


sklearn版のX_trainデータ数：1027
sklearn版のy_trainデータ数：1027


scikit-learnを使ったコードを作成していきます。<br>
<br>
検証データの分割には問題1で作成した自作の関数を用いてください。クロスバリデーションではなくホールドアウト法で構いません。

#### 分類問題

分類は3種類の手法をスクラッチします。<br>

- ロジスティック回帰
- SVM
- 決定木<br>

<br>
ロジスティック回帰はscikit-learnにおいてLogisticRegressionクラスとSGDClassifierクラスの2種類から使用できます。ここでは勾配降下法を用いて計算するSGDClassifierクラスを利用してください。引数でloss="log"とすることでロジスティック回帰の計算になります。<br>
<br>
データセットは3種類用意します。<br>
<br>
1つ目は事前学習期間同様にirisデータセットです。<br>
<br>
2値分類としたいため、以下の2つの目的変数のみ利用します。特徴量は4種類全て使います。<br>
<br>
- virgicolorとvirginica<br>

<br>
残り2つは特徴量が2つのデータセットを人工的に用意します。以下のコードで説明変数X,目的変数yが作成可能です。「シンプルデータセット1」「シンプルデータセット2」とします。特徴量が2つであるため可視化が容易です。

In [104]:
# シンプルデータセット1作成コード

np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, int(n_samples/2))
f1 = np.random.multivariate_normal(f1, cov, int(n_samples/2))
X1 = np.concatenate((f0, f1))
y1 = np.concatenate((np.ones((int(n_samples/2))), np.ones((int(n_samples/2))) *(-1))).astype(np.int)
random_index = np.random.permutation(np.arange(n_samples))
X1 = X1[random_index]
y1 = y1[random_index]

In [94]:
# シンプルデータセット2作成コード

X2 = np.array([[-0.44699 , -2.8073  ],[-1.4621  , -2.4586  ],
           [ 0.10645 ,  1.9242  ],[-3.5944  , -4.0112  ],
           [-0.9888  ,  4.5718  ],[-3.1625  , -3.9606  ],
           [ 0.56421 ,  0.72888 ],[-0.60216 ,  8.4636  ],
           [-0.61251 , -0.75345 ],[-0.73535 , -2.2718  ],
           [-0.80647 , -2.2135  ],[ 0.86291 ,  2.3946  ],
           [-3.1108  ,  0.15394 ],[-2.9362  ,  2.5462  ],
           [-0.57242 , -2.9915  ],[ 1.4771  ,  3.4896  ],
           [ 0.58619 ,  0.37158 ],[ 0.6017  ,  4.3439  ],
           [-2.1086  ,  8.3428  ],[-4.1013  , -4.353   ],
           [-1.9948  , -1.3927  ],[ 0.35084 , -0.031994],
           [ 0.96765 ,  7.8929  ],[-1.281   , 15.6824  ],
           [ 0.96765 , 10.083   ],[ 1.3763  ,  1.3347  ],
           [-2.234   , -2.5323  ],[-2.9452  , -1.8219  ],
           [ 0.14654 , -0.28733 ],[ 0.5461  ,  5.8245  ],
           [-0.65259 ,  9.3444  ],[ 0.59912 ,  5.3524  ],
           [ 0.50214 , -0.31818 ],[-3.0603  , -3.6461  ],
           [-6.6797  ,  0.67661 ],[-2.353   , -0.72261 ],
           [ 1.1319  ,  2.4023  ],[-0.12243 ,  9.0162  ],
           [-2.5677  , 13.1779  ],[ 0.057313,  5.4681  ]])
y2 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## 【問題2】 分類問題を解くコードの作成
上記3種類の手法で3種類のデータセットを学習・推定するコードを作成してください。

In [1]:
# 各分類器をimport
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [145]:
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris()

data = pd.DataFrame(data=iris.data, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
target = pd.DataFrame(data=iris.target, columns=['Species'])

df = pd.concat([data, target], axis=1)
df = df.loc[df['Species'] !=0, ['sepal_length', 'petal_length', 'Species']]

X = np.array(df.iloc[:, :-1])
y = np.array(df.iloc[:, -1])

In [152]:
log_reg = SGDClassifier(loss='log')
svc = SVC(probability=True)
tree = DecisionTreeClassifier()

database = [[X, y],
            [X1, y1],
            [X2, y2]]
d_labels = ['iris', 'samp1', 'samp2']
models = [log_reg, svc, tree]
m_labels = ['Logistic Regression', 'SVC', 'Decision Tree']

for model, m_label in zip(models, m_labels):
    print(m_label)
    
    for data, d_label in zip(database, d_labels):

        X_train, X_test, y_train, y_test = scratch_train_test_split(data[0], data[1], train_size=0.7)
        
        model.fit(X_train, y_train)
        pred = model.predict_proba(X_test)

        print("{} data - ROC AUC score: {:.5f}"
              .format(d_label, roc_auc_score(y_test, pred[:,1])))
    print("\n")

Logistic Regression
iris data - ROC AUC score: 0.98222
samp1 data - ROC AUC score: 1.00000
samp2 data - ROC AUC score: 0.73611


SVC
iris data - ROC AUC score: 1.00000
samp1 data - ROC AUC score: 1.00000
samp2 data - ROC AUC score: 0.52778


Decision Tree
iris data - ROC AUC score: 0.83333
samp1 data - ROC AUC score: 1.00000
samp2 data - ROC AUC score: 0.41667




#### 回帰問題
回帰は1種類をスクラッチします。<br>

- 線形回帰<br>

<br>
線形回帰は勾配降下法を用いて計算するSGDRegressorクラスを利用してください。<br>
データセットは事前学習期間同様にHouse Pricesコンペティションのものを使います。<br>
<br>
train.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。

## 【問題3】 回帰問題を解くコードの作成
線形回帰でHouse Pricesデータセットを学習・推定するコードを作成してください。

In [155]:
home_data = pd.read_csv('/Users/tamiyagt/Documents/machine learning/02_Kaggle/house prices/train.csv')
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

#GrLivArea、YearBuilt、SalePriceを抽出
train = home_data.loc[:, ['GrLivArea', 'YearBuilt', 'SalePrice']]

train.head()

,GrLivArea,YearBuilt,SalePrice
0,1710,2003,208500
1,1262,1976,181500
2,1786,2001,223500
3,1717,1915,140000
4,2198,2000,250000


In [157]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

sgdr = SGDRegressor()

X = np.array(train.iloc[:, :-1])
y = np.array(train.iloc[:, -1])

X_train, X_test, y_train, y_test = scratch_train_test_split(X, y, train_size=0.7, stratify=False)

# 標準化処理
scaler = StandardScaler()

X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

sgdr.fit(X_train_std, y_train)
pred = sgdr.predict(X_test_std)

sgdr_mse = mean_squared_error(pred, y_test)

print("SGDRegressor MSE：{:.3g}".format(sgdr_mse))

SGDRegressor MSE：2.53e+09
